## Numerical Analysis - Fall semester 2024
# Serie 02 - Introduction to non-linear equations

First, we will need to import some of the usual packages. You will have to run this cell every time you restart your notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

<hr style="clear:both">

### Bisection method for a problem in statics

A certain planar mechanical system consists of four rigid rods of length $a_1$, $a_2$, $a_3$, and $a_4$ linked together. The configuration in which this rods are in with respect to each other is uniquely determined by the angles between the first and second rod, denoted with $\theta$, and the first and fourth rod, denoted with $\omega$. We provide you with a function which visualizes such a configuration. Run the below two cells to see the result.

In [ ]:
def plot_configuration(theta, omega, a1, a2, a3, a4):
    fig, ax = plt.subplots()
    ax.axis("equal")
    ax.axis("off")
    ax.plot([- a1 / 3, 4 * a1 / 3], [0, 0], linewidth=4, linestyle="--", color="black")
    ax.plot([0, np.cos(theta) * a2, a1 + np.cos(omega) * a3, a1, 0], [0, np.sin(theta) * a2, np.sin(omega) * a3, 0, 0], linewidth=7, marker="o", markersize=15, color="black")
    ax.text(a1 / 2, 0, "$a_1$", fontsize=20, horizontalalignment="center", verticalalignment="center", bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.1'))
    ax.text(np.cos(theta) * a2 / 2, np.sin(theta) * a2 / 2, "$a_2$", fontsize=20, horizontalalignment="center", verticalalignment="center", bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.1'))
    ax.text(a1 + np.cos(omega) * a3 / 2, np.sin(omega) * a3 / 2, "$a_3$", fontsize=20, horizontalalignment="center", verticalalignment="center", bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.1'))
    ax.text((np.cos(theta) * a2 + a1 + np.cos(omega) * a3) / 2, (np.sin(theta) * a2 + np.sin(omega) * a3) / 2, "$a_4$", fontsize=20, horizontalalignment="center", verticalalignment="center", bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.1'))
    ax.add_patch(matplotlib.patches.Arc((a1, 0), a1 / 3, a1 / 3, theta1=0, theta2=omega * 180 / np.pi, linewidth=4))
    ax.text(6 * a1 / 5, 0, r"$\omega$", fontsize=20, horizontalalignment="left", verticalalignment="bottom")
    ax.add_patch(matplotlib.patches.Arc((0, 0), a1 / 3, a1 / 3, theta1=0, theta2=theta * 180 / np.pi, linewidth=4))
    ax.text(a1 / 5, 0, r"$\theta$", fontsize=20, horizontalalignment="left", verticalalignment="bottom")
    plt.show()

In [ ]:
theta = np.pi / 4
omega = np.pi / 2
a1 = 1
a2 = 1
a3 = 1 / np.sqrt(2)
a4 = 1 - 1 / np.sqrt(2)
plot_configuration(theta, omega, a1, a2, a3, a4)

Given the lengths of the rods $a_1$, $a_2$, $a_3$, $a_4$, and the angle $\omega$, we would like to find out what value the angle $\theta$ takes. In the lecture you have seen that every valid configuration satisfies the non-linear equation: 

$$
\frac{a_1}{a_2} \cos(\omega) - \frac{a_1}{a_4} \cos(\theta) - \cos(\omega - \theta) + \frac{a_1^2 + a_2^2 - a_3^2 + a_4^2}{2 a_2 a_4} = 0
$$

<div class="alert alert-success">
    
**Exercise 1:** Complete the function `constraint`, which returns the value of the left-hand side of the above equation for given $\theta$, $\omega$, $a_1$, $a_2$, $a_3$, and $a_4$.

*Hint*: You can use `np.cos` to compute the cosine of a number.
</div>


In [ ]:
def constraint(theta, omega, a1, a2, a3, a4):
    """ constraint function """

    ### BEGIN SOLUTION
    return a1/a2 * np.cos(omega) - a1/a4 * np.cos(theta) - np.cos(omega - theta) + (a1**2 + a2**2 - a3**2 + a4**2) / (2*a2*a4)
    ### END SOLUTION

In [ ]:
assert not (constraint(np.pi / 2, np.pi / 2, 1, 1, 1, 1) is None), f"the function 'constraint' returned nothing, make sure to 'return' your result"; assert (C := constraint(np.pi / 2, np.pi / 2, 1, 1, 1, 1)) == 0.0, f"'constraint(np.pi / 2, np.pi / 2, 1, 1, 1, 1)' should return 0 but got {C}"; assert (C := constraint(np.pi / 2, np.pi / 2, 1, 1, 1, 1)) == 0.0, f"'constraint(np.pi / 4, np.pi / 4, 1, 1, 1, 1)' should return 0 but got {C}"; assert (C := constraint(np.pi / 2, 0, 2, 1, 1, 1)) == 4.5, f"'constraint(np.pi / 2, 0, 1, 1, 1, 1)' should return 4.5 but got {C}"; print("Great job! Your implementation passes our checks.")

As you have seen during the lecture, this non-linear equation cannot be solved analytically. Hence, we will need to solve this nonlinear equation analytically.

<div class="alert alert-success">
    
**Exercise 2:** In the below cell, complete the `bisection` function which takes as input a function $f:\mathbb{R} \to \mathbb{R}$, the startpoint `a` and endpoint `b` of the search-interval $[a, b]$, and a tolerance `tol`, and uses the bisection method (Algorithm 1.2 in the lecture notes) to output the location of a zero `alpha` and the number of iterations `niter`.

*Hint*: Additionally to Algorithm 1.2, our implementation also checks in every iteration if any of $f(a)$, $f(b)$, or $f(x^{(k)})$ are zero, in which case the root has already been found!
</div>


In [ ]:
def bisection(f, a, b, tol):
    """ bisection method with stopping criterion """

    alpha = a  # approximate root
    k_min = int(np.ceil(np.log2((b - a) / tol) - 1))  # compute number of iterations needed
    x_k = (a + b) / 2
    for k in range(k_min):
        if f(x_k) == 0:
            return x_k, k
        elif f(a) == 0:
            return a, k
        elif f(b) == 0:
            return b, k
        
        ### BEGIN SOLUTION
        if f(x_k) * f(a) < 0:
            b = x_k
        else:
            a = x_k
        x_k = (a + b) / 2
        ### END SOLUTION

    alpha = x_k
    niter = k + 1
    
    return alpha, niter

In [ ]:
assert not (bisection(lambda x: x, -np.pi, 1, 0.1) is None), f"the function 'bisection' returned nothing, make sure to 'return' your result"; assert isinstance(bisection(lambda x: x, -np.pi, 1, 1e-10), tuple), f"'bisection' should return 2 items, but only got 1"; assert np.isclose(B := bisection(lambda x: x, -1e-10, 1, 1e-10)[0], 0), f"'bisection(f, -1e-10, 1, 1e-10)' for 'f(x) = x' should return 'alpha = 0', but got {B}"; assert np.isclose(B := bisection(lambda x: x, -1, 1e-10, 1e-10)[0], 0), f"'bisection(f, -1, 1e-10, 1e-10)' for 'f(x) = x' should return 'alpha = 0', but got {B}"; assert (B := bisection(lambda x: x, -1e-10, 1, 0.5)[1]) == 1, f"'bisection(f, -1e-10, 1, 0.5)' for 'f(x) = x' should return 'niter = 1', but got {B}"; print("Great job! It seems like your function works correctly.")

Now, we would like to see how our method works in practice. We take a simple configuration where all rods are of length $1$ and $\omega = \pi / 4$. We will use the bisection method to look for a valid $\theta \in [0, \pi]$ and plot the result below.

In [ ]:
a1 = 1
a2 = 1
a3 = 1
a4 = 1
omega = np.pi / 4

target = lambda theta: constraint(theta, omega, a1, a2, a3, a4)
theta, _ = bisection(target, 0, np.pi, 1e-10)
print(f"found valid configuration with angle theta = {theta}")
plot_configuration(theta, omega, a1, a2, a3, a4)

Of course $\theta = 0$ is a valid configuration, where the first and second, and the third and fourth rod overlap. However, if we suppose the rods cannot overlap, then we will need to exclude $\theta = 0$ from our search interval. We will do this by slightly shrinking the interval to $[10^{-10}, \pi]$.

In [ ]:
a1 = 1
a2 = 1
a3 = 1
a4 = 1
omega = np.pi / 4

target = lambda theta: constraint(theta, omega, a1, a2, a3, a4)
theta, _ = bisection(target, 1e-10, np.pi, 1e-10)
print(f"found valid configuration with angle theta = {theta}")
plot_configuration(theta, omega, a1, a2, a3, a4)

And now a bit more interesting configuration. The angle $\omega$ is now larger than $\pi / 2$, while all rods are of length $1$, except for the first one which is half that lenght. Feel free to play around with the parameters a bit with this example. Be aware, that many combinations of parameters will be invalid, in which case often $\theta = \pi$ is returned.

In [ ]:
a1 = 0.5
a2 = 1
a3 = 1
a4 = 1
omega = 0.75 * np.pi

target = lambda theta: constraint(theta, omega, a1, a2, a3, a4)
theta, _ = bisection(target, 0, np.pi, 1e-10)
print(f"found valid configuration with angle theta = {theta}")
plot_configuration(theta, omega, a1, a2, a3, a4)

Only few configurations of parametrs are valid. For example, when the length of one rod is larger than the sum of the lengths of the other rods. Then we can see that the `constraint` function will never be zero. We can already see this by plotting it as a function of $\theta$.

In [ ]:
a1 = 1
a2 = 1
a3 = 1
a4 = 4
omega = np.pi / 4
x_lin = np.linspace(0, np.pi)
target = lambda theta: constraint(theta, omega, a1, a2, a3, a4)
plt.plot(x_lin, target(x_lin))
plt.xlabel(r"$\theta$")
plt.ylabel("constraint function")
plt.grid()
plt.show()

<hr style="clear:both">

### Behavior of bisection method

We are interested in the roots of the function

$$f(x) = \frac{x}{2} - \sin(x) + \frac{\pi}{6} - \frac{\sqrt{3}}{2}.$$


<div class="alert alert-success">
    
**Exercise 3:** Define a Python function `f` which returns $f(x)$ for a given real number $x$ and use it to plot the graph of the function $f(x)$ at $100$ uniformly spaced points in the interval $[-\pi/2,\pi]$. 
</div>


In [ ]:
def f(x):
    ### BEGIN SOLUTION
    return x / 2 - np.sin(x) + np.pi / 6 - np.sqrt(3) / 2
    ### END SOLUTION
    
### BEGIN SOLUTION
x_lin = np.linspace(-np.pi / 2, np.pi, 100)
plt.xlabel(r"$x$")
plt.ylabel(r"$f(x)$")
plt.plot(x_lin, f(x_lin))
plt.axhline(0, x_lin[0], x_lin[-1], linestyle="--", color="gray")
plt.grid()
plt.show()
### END SOLUTION

<div class="alert alert-success">

**Exercise 4:** How many roots does the function $f$ have? For which roots can we apply the bisection method? Why?
</div>

=== BEGIN MARK SCHEME ===

There are two roots: one close to $-1$ (refer to it as $\alpha_1$) and one close to $2$ (refer to it as $\alpha_2$). The bisection method can be used to compute the root $\alpha_2$, but we cannot use it to compute $\alpha_1$, as the condition $f(a) f(b) < 0$ is not satisfied around $\alpha_1$.

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 5:** Estimate the minimal number of iterations the bisection method needs to compute the root within the interval $[\pi/2, \pi]$ with a tolerance of $10^{-10}$.
</div>

=== BEGIN MARK SCHEME ===

We consider the second root $\alpha_2 \in I_2 = (\pi/2, \pi)$ and we apply the bisection method to find an approximate value, with a tolerance of $tol=10^{-10}$. To estimate the necessary number of iterations, we can write:
\begin{equation*}
  \vert e^{(k)} \vert = \vert x^{(k)} - \alpha_2 \vert \leq \frac{b-a}{2^{k+1}} \; ,
\end{equation*}
with $a=\pi/2$, $b=\pi$ and impose the condition $(b-a) / 2^{k+1} \leq 10^{-10}$. We have:
\begin{equation*}
(k+1)\log(2)\ge \log \left(\frac{b-a}{tol}\right)
\end{equation*}
and in our case:
\begin{equation*}
k\ge \log \left(\frac{\pi/2}{10^{-10}}\right)/\log(2)-1 \approx 32.87
\end{equation*}
Therefore, we find that $k\geq 33$ iterations are necessary.

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 6:** Check your answer to Exercise 5 by using your implementation of the function `bisection` from Exercise 2 and verify that the found solution satisfies the tolerance $10^{-10}$. 
    
*Hint:* You may use that $x_{\mathrm{true}}=2.246005589297974$ is a good approximation to the true root.
</div>


In [ ]:
x_true = 2.246005589297974

### BEGIN SOLUTION
alpha, niter = bisection(f, np.pi / 2, np.pi, 1e-10)
print(f"number of iterations: {niter}")
print(f"absolute distance from the true zero: {abs(alpha - x_true)}")
### END SOLUTION

<hr style="clear:both">

### Convergence of bisection method

Let us consider the function represented in the figure which appears when you run the cell below.

In [ ]:
x_lin = np.linspace(-2, 2, 100)
f = lambda x: 1/3 - x/3 - x**2/3 + x**3/3
plt.plot(x_lin, f(x_lin))
plt.scatter([-1, 1], [0, 0], s=50, color="black")
plt.text(-1, 0, r"$\alpha$", fontsize=20, horizontalalignment="right", verticalalignment="bottom")
plt.text(1, 0, r"$\beta$", fontsize=20, horizontalalignment="left", verticalalignment="top")
plt.grid()
plt.show()

<div class="alert alert-success">
    
**Exercise 7:** If we use the bisection method to find one of the roots of the function using $[-2, 2]$ as the initial interval and $10^{-5}$ as the tolerance, to which root does the method converge and in how many iterations? [Choose one answer]

1. The method converges to $\alpha$ in at most $18$ iterations.
2. The method converges to $\beta$ in at most $18$ iterations.
3. The method converges to $\alpha$ in at most $10$ iterations. 
4. The method converges to $\beta$ in at most $10$ iterations. 
5. It is not possible to tell to which root the method will converge, but we will need at most $18$ iterations.
</div>

=== BEGIN MARK SCHEME ===

Answer 1 is the correct choice.

The method cannot converge to $\beta$, as the function doesn't change signs around $\beta$. However, the function changes signs around $\alpha$, therefore the method converges to $\alpha$.

Using the formula
$$
k\ge \log_2 \left(\frac{b-a}{tol}\right)-1 
$$
we find that we need at most $18$ iterations.

=== END MARK SCHEME ===

<hr style="clear:both">

### Analysis of population growth with fixed point method

We consider a population of size $x^{(k)}$ over mulltiple generations $k = 1, 2, \dots$. The growth of this population is modelled by a function $\phi : \mathbb{N} \to \mathbb{N}$, which, for the $k$-th generation's population size $x^{(k)}$, predicts the $(k+1)$-th population size $x^{(k+1)}$. We consider three models for this

- Malthus: $\phi_1(x) = 2x$
- Verhulst: $\phi_2(x) = 2x / (1 + x)$
- Prey-predator: $\phi_3(x) = 2x^2 / (1 + x^2)$

<div class="alert alert-success">
    
**Exercise 8:** Define Python functions corresponding to each of the above population growth models.
</div>


In [ ]:
def phi_1(x):
    """ Malthus population growth model """

    ### BEGIN SOLUTION
    return 2 * x
    ### END SOLUTION

def phi_2(x):
    """ Verhulst population growth model """

    ### BEGIN SOLUTION
    return 2 * x / (1 + x)
    ### END SOLUTION

def phi_3(x):
    """ Prey-predator population growth model """

    ### BEGIN SOLUTION
    return 2 * x ** 2 / (1 + x ** 2)
    ### END SOLUTION

<div class="alert alert-success">
    
**Exercise 9:** Visualize each of these models by plotting the function on 100 uniformly spaced points in the interval $I=[0, 2]$, and try to identify visually the fixed-points $\alpha = \phi(\alpha)$ of these models by adding the line $y = x$ to them.
</div>


In [ ]:
### BEGIN SOLUTION
x_lin = np.linspace(0, 2, 100)
plt.plot(x_lin, phi_1(x_lin), label=r"$\phi_1$ (Malthus)")
plt.plot(x_lin, phi_2(x_lin), label=r"$\phi_2$ (Verhulst)")
plt.plot(x_lin, phi_3(x_lin), label=r"$\phi_3$ (Prey-predator)")
plt.plot(x_lin, x_lin, linestyle="--", color="gray", label=r"$x = y$")
plt.legend()
plt.grid()
plt.show()
### END SOLUTION

Below, we provide you with an implementation of the fixed-point method (Algorithm 1.4 in the lecture notes).

In [ ]:
def fixed_point(phi, x0, tol, nmax):
    """
    Fixed point iterations.

    Parameters
    ----------
    phi : callable
        The function representing the fixed point iteration.
    x0 : float
        The initial guess for the fixed point.
    tol : float
        The desired tolerance for the fixed point.
    nmax : int
        The maximum number of iterations.

    Returns
    -------
    x_seq : array-like
        The successive values of the fixed point iterations.
    res : array-like
        The value of the residual at each iteration.
    niter : int
        The number of iterations performed.
    """

    niter = 0
    x_seq = []
    x_seq.append(x0)

    xt = phi(x0)
    res = []
    res.append(x0 - xt)  # this measures ``how much x0 is far from the fixed point''

    while (abs(res[-1]) > tol) and (niter < nmax):
        niter = niter + 1
        x_seq.append(xt)
        x0 = xt
        xt = phi(xt)
        res.append(abs(x0 - xt))

    if niter >= nmax:
        print(
            [
                "fixedPoint stopped without converging to the desired\n "
                "tolerance because the maximum number of iterations was reached\n"
            ]
        )

    # convert from list to array
    x_seq = np.array(x_seq)
    res = np.array(res)

    return x_seq, res, niter

<div class="alert alert-success">
    
**Exercise 10:** Let us write as $e^{(k)}=\vert x^{(k)}-\alpha \vert$ the error at the $k^{\text{th}}$ iteration and $r^{(k)}=\vert x^{(k)}-\phi_i(x^{(k)})\vert$ the residual. For the models $\phi_2$ and $\phi_3$, plot $e^{(k)}$ and $r^{(k)}$ in terms of $k$ for the fixed-point $\alpha=1$ in a semi-logarithmic graph (use the command `plt.semilogy`). Use the function `fixed_point` with an initial value $x^{(0)} = 2$, a tolerance of $10^{-6}$, and at most $2000$ iterations.
</div>


In [ ]:
### BEGIN SOLUTION
x_seq_2, res_2, niter_2 = fixed_point(phi_2, 2, 1e-6, 2000)
err_2 = np.abs(x_seq_2 - 1)
x_seq_3, res_3, niter_3 = fixed_point(phi_3, 2, 1e-6, 2000)
err_3 = np.abs(x_seq_3 - 1)

plt.semilogy(res_2, label=r"r^{(k)}")
plt.semilogy(err_2, label=r"e^{(k)}")
plt.xlabel("k")
plt.grid()
plt.show()

plt.semilogy(res_3, label=r"r^{(k)}")
plt.semilogy(err_3, label=r"e^{(k)}")
plt.xlabel("k")
plt.grid()
plt.show()
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 11:** Compute the derivative $\phi_2'$ of $\phi_2$ and evaluate it at $1$. Use your result to justify the progression of the error in your plot from Exercise 10.

*Hint:* Make use of Theorem 1.1 from the lecture notes.
</div>

=== BEGIN MARK SCHEME ===

With the quotient rule we obtain
$$
\phi_2'(x) = \frac{2}{(1 + x)^2}.
$$
Therefore, $\phi_2'(1) = 0.5$. By Theorem 1.1 we can expect the error to half in every iteration when we are close to the fixed point. As a consequence, we see a linear decay of the error in the semi-logarithmic plot in Exercise 10.

=== END MARK SCHEME ===

<hr style="clear:both">

## The end

Congratulations! You have made it to the end of the second exercise notebook. 